# Our API Code


In [1]:
import json
import os
from pathlib import Path
from typing import List, Tuple, Dict, Any

import numpy as np
import pandas as pd
import json
import pandas as pd

import json
import pandas as pd
import glob

from pandas import DataFrame


class KabaddiDataAPI:
    def __init__(self):
        self.base_path = "./MatchData_pbp"
    # for a season - display the standings

    def create_team_info(self, team, season, group_name):
        """Create a dictionary with team information for a given season and group."""

        return {
            'Group': group_name,
            'Season': season,
            'Team_Name': team['team_name'],
            'Team_Id': team['team_id'],
            'team_short_name': team['team_short_name'],
            'League_position': team['position'],
            'Matches_played': team['played'],
            'Wins': team['wins'],
            'Lost': team['lost'],
            'Tied': team['tied'],
            'Draws': team['draws'],
            'No Result': team['noresult'],
            'League_points': team['points'],
            'Score_diff': team['score_diff'],
            'Qualified': team['is_qualified'],
        }

    def create_matches_list(self, team, group_name):
        matches = []
        for match in team['match_result']['match']:
            match_info = {
                'Group': group_name,
                'match_id': match['id'],
                'date': match['date'],
                'teama_id': match['teama_id'],
                'result': match['result'],
                'teama_short_name': match['teama_short_name'],
                'teama_score': match['teama_score'],
                'teamb_id': match['teamb_id'],
                'teamb_short_name': match['teamb_short_name'],
                'teamb_score': match['teamb_score'],
                'match_result': match['match_result']
            }

            print(match_info)
            matches.append(match_info)
        return matches

    def process_matches(self, matches_list):
        matches_df = pd.DataFrame(matches_list)
        matches_df = matches_df[(matches_df['result'].isin(['W', 'T'])) | (matches_df['result'].isnull())]
        matches_df = matches_df.sort_values(by='date', ascending=True)
        matches_df = matches_df.set_index('match_id').rename_axis('match id')
        return matches_df

    def get_pkl_standings(self, season=None, qualified=False, team_id=None, matches=False):

        if season is None:
            season = 10

        file_path = Path(f"./PKL_Standings/pkl_standings_s{season}.json")

        with open(file_path, 'r') as f:
            data = json.load(f)

        standings = data['standings']

        team_standings_info_list, qualified_teams_info_df, matches_list, = [], [], []

        season_name = standings['series_name']
        champion_team_id = standings['champion_id']

        if len(standings['groups']) == 0:
            return pd.DataFrame()

        if len(standings['groups']) == 1:

            group = standings['groups'][0]
            if 'name' in group and group['name'] != "":
                group_name = group['name']
            else:
                group_name = 'Main'

            for team in group['teams']['team']:
                if team_id is None or int(team['team_id']) == team_id:
                    team_standings_info = {
                        'Group': group_name,
                        'Season': season,
                        'Team_Id': team['team_id'],
                        'Team_Name': team['team_name'],
                        # 'team_short_name': team['team_short_name'],
                        'League_position': team['position'],
                        'Matches_played': team['played'],
                        'Wins': team['wins'],
                        'Lost': team['lost'],
                        'Tied': team['tied'],
                        'Draws': team['draws'],
                        'No Result': team['noresult'],
                        'League_points': team['points'],
                        'Score_diff': team['score_diff'],
                        'Qualified': team['is_qualified'],
                    }

                    if qualified and team['is_qualified']:
                        qualified_teams_info_df.append(team_standings_info)

                    team_standings_info_list.append(team_standings_info)

                    if matches:
                        matches_list.extend(self.create_matches_list(team, group_name))

        else:

            for group in standings['groups']:

                if 'name' in group and group['name'] != "":
                    group_name = group['name']
                else:
                    group_name = 'Main'

                for team in group['teams']['team']:
                    if team_id is None or int(team['team_id']) == team_id:
                        team_info = {
                            'Group': group_name,
                            'Season': season,
                            'Team_Id': team['team_id'],
                            'Team_Name': team['team_name'],
                            # 'team_short_name': team['team_short_name'],
                            'League_position': team['position'],
                            'Matches_played': team['played'],
                            'Wins': team['wins'],
                            'Lost': team['lost'],
                            'Tied': team['tied'],
                            'Draws': team['draws'],
                            'No Result': team['noresult'],
                            'League_points': team['points'],
                            'Score_diff': team['score_diff'],
                            'Qualified': team['is_qualified'],
                        }

                        if qualified and team['is_qualified']:
                            qualified_teams_info_df.append(team_info)

                        team_standings_info_list.append(team_info)

                        if matches:
                            matches_list.extend(self.create_matches_list(team, group_name))

        team_info_df = pd.DataFrame(team_standings_info_list)

        if qualified:
            qualified_teams_df = pd.DataFrame(qualified_teams_info_df)
            return qualified_teams_df, team_info_df

        if matches:
            matches_df = self.process_matches(matches_list)
            return team_info_df, matches_df

        else:
            return team_info_df

    def get_season_matches(self, season="all"):
        matches_list = []

        # Determine the file(s) to load based on the season input
        if season == "all":
            files = glob.glob('./Matches-Overview/S*_PKL_MatchData.json')
        else:
            files = [f'./Matches-Overview/S{season}_PKL_MatchData.json']

        for file in files:
            with open(file) as f:
                data = json.load(f)

            # Loop over each match in the file
            for match in data['matches']:
                match_details = {
                    "Match Name": match['event_name'],
                    'Match ID': match['game_id'],
                    "Tour Name": match['tour_name'],
                    "Venue": match['venue_name'],
                    'Match_Outcome': match['event_sub_status'],
                    "Date": match['start_date'],
                    "Result": match['event_sub_status'],
                    "Winning Margin": match['winning_margin']
                }

                for participant in match['participants']:
                    match_details[f"{participant['name']} Score"] = participant['value']

                matches_list.append(match_details)

        # Convert the list of dictionaries into a DataFrame
        df = pd.DataFrame(matches_list)

        #df.to_csv("matches_data.csv", index=False)

        # Display the DataFrame
        return df

    def get_team_matches(self,season,team_id :str):
        matches_list = []

        # Determine the file(s) to load based on the season input
        if season == "all":
            files = glob.glob('./Matches-Overview/S*_PKL_MatchData.json')
        else:
            files = [f'./Matches-Overview/S{season}_PKL_MatchData.json']

        for file in files:
            with open(file) as f:
                data = json.load(f)

            # Loop over each match in the file
            for match in data['matches']:
                if match['participants'][0]['id']==team_id or  match['participants'][1]['id']==team_id:
                    match_details = {
                        "Match Name": match['event_name'],
                        'Match ID': match['game_id'],
                        "Tour Name": match['tour_name'],
                        "Venue": match['venue_name'],
                        'Match_Outcome': match['event_sub_status'],
                        "Date": match['start_date'],
                        "Result": match['event_sub_status'],
                        "Winning Margin": match['winning_margin']
                    }
                    matches_list.append(match_details)
            df = pd.DataFrame(matches_list)
            return df
    def get_team_info(self, team_id, season='overall'):

        if season != 'overall':
            season = int(season)
        df = pd.read_csv("../prokabaddidata/Team-Wise-Data/PKL_AggregatedTeamStats.csv")

        df['team_id'] = pd.to_numeric(df['team_id'], errors='coerce')
        df['season'] = pd.to_numeric(df['season'], errors='coerce')

        team_id = int(team_id)

        if season == 'overall':
            all_row = df[(df['team_id'] == team_id) & (df['season'] == 'all')]
            other_rows = df[(df['team_id'] == team_id) & (df['season'] != 'all')]
            filtered_df = pd.concat([all_row, other_rows]).reset_index(drop=True)
        else:
            filtered_df = df[(df['team_id'] == team_id) & (df['season'] == season)]

        if filtered_df.empty:
            print(f"No data found in CSV for team_id {team_id} in season {season}")
            return None, None, None, None, None
        if season == 'overall':
            standings_df, matches_df = self.get_pkl_standings(season=10, team_id=team_id, matches=True)
        else:
            standings_df, matches_df = self.get_pkl_standings(season, team_id=team_id, matches=True)
        standings_df = None
        matches_df = None
        # Separate the columns based on the suffixes
        rank_columns = [col for col in filtered_df.columns if col.endswith('_rank')]
        value_columns = [col for col in filtered_df.columns if col.endswith('_value')]
        per_match_columns = [col for col in filtered_df.columns if col.endswith('_per-match')]

        df_rank = filtered_df[['season', 'team_id', 'team_name'] + rank_columns]
        df_value = filtered_df[['season', 'team_id', 'team_name'] + value_columns]
        df_per_match = filtered_df[['season', 'team_id', 'team_name'] + per_match_columns]

        return df_rank, df_value, df_per_match, standings_df, matches_df

    def get_player_info(self, player_id, season=None):
        player_id = int(player_id)
        file_path = "./Player-Wise-Data/all_seasons_player_stats_rounded.csv"
        df = pd.read_csv(file_path)
        file_rvd = r"./Player-Wise-Data/merged_raider_v_num_defenders_FINAL.csv"
        rvd_df = pd.read_csv(file_rvd)

        # raid_file = "./Player-Wise-Data/AllSeasons_AllTeams_RaiderSuccessRate.csv"
        # raid_df = pd.read_csv(raid_file)

        defend_file = "./Player-Wise-Data/AllSeasons_AllTeams_DefenderSuccessRate.csv"
        defend_df = pd.read_csv(defend_file)

        def to_numeric_or_nan(x):
            try:
                return pd.to_numeric(x)
            except ValueError:
                return np.nan

        df['player_id'] = df['player_id'].apply(to_numeric_or_nan)
        df['player_id'] = df['player_id'].fillna(-1)
        df['player_id'] = df['player_id'].astype(np.int64)

        rvd_df['player-id'] = rvd_df['player-id-pkdc-sanitised'].apply(to_numeric_or_nan)
        rvd_df['player-id'] = rvd_df['player-id'].fillna(-1)
        rvd_df['player-id'] = rvd_df['player-id'].astype(int)


        defend_df['player_id'] = defend_df['player_id_copy_backup'].apply(to_numeric_or_nan)
        defend_df['player_id'] = defend_df['player_id'].fillna(-1)
        defend_df['player_id'] = defend_df['player_id'].astype(np.int64)

        # If season is not specified, use the latest season
        if season is None:
            season = df['season'].max()

        # Player aggregated stats
        player_stats_df = df[(df['player_id'] == player_id) & (df['season'] == season)]

        # Raiders v defenders
        rvd_data = rvd_df[rvd_df['player-id'] == player_id]
        if rvd_data.empty:
            print(f"No data for raiders v no of defenders for {player_id}")
        rvd_extracted_df = rvd_data[rvd_data['season'].str.extract(r'(\d+)')[0].astype(int) == season]

        defend_extracted_df = defend_df[(defend_df['player_id'] == player_id) & (defend_df['season'] == season)]

        if player_stats_df.empty:
            print(f"Data not available for player_id {player_id} for season {season}")

        if rvd_extracted_df.empty:
            print(f"Raiders vs defenders data not available for player_id {player_id} for season {season}")
        if defend_extracted_df.empty:
            print(f"successful defender rate not available for player_id {player_id} for season {season}")

        return player_stats_df, rvd_extracted_df, defend_extracted_df

    def build_team_roster(self,team_id, season_number):
        roster = []  # Initialize the roster list

        for i in os.listdir("./MatchData_pbp"):
            if f"Season_{season_number}" in i:
                break

        directory_path = os.path.join("./MatchData_pbp", i)

        # print(f"Starting to build the roster for Team ID: {team_id}, Season: {season_number}...\n")

        # Iterate over all files in the directory
        for filename in os.listdir(directory_path):
            if filename.endswith(".json"):  # Process only JSON files
                file_path = os.path.join(directory_path, filename)
                # print(f"Loading file: {filename}")

                with open(file_path, 'r') as f:
                    match_data = json.load(f)
                    if season_number == 4:

                        # Check if the match belongs to the specified season
                        if match_data['match_detail']['series']['id'] == season_number:

                            # print(f"Processing match ID: {match_data['match_detail']['match_id']}")

                            teams = match_data['teams']['team']
                            for team in teams:
                                if team['id'] == team_id:
                                    # print(f"Found team {team['name']} (ID: {team_id}) in the match.")
                                    squad = team['squad']
                                    for player in squad:
                                        player_id = player['id']
                                        player_name = player['name']

                                        # Check if player is already in the roster
                                        if player_id not in [p['Player ID'] for p in roster]:
                                            player_details = {
                                                'Player ID': player_id,
                                                'Name': player_name,
                                                'Jersey Number': player.get('jersey', None),
                                                'Captain': player.get('captain', False),
                                                'Played': player.get('played', False),
                                                'Starter': player.get('starter', False),
                                                'Top Raider': player.get('top_raider', False),
                                                'Top Defender': player.get('top_defender', False)
                                            }

                                            roster.append(player_details)
                    else:
                        series_dict = {
                            10: '44',
                            9: '25',
                            8: '20',
                            7: '11',
                            6: '10',
                            5: '8',
                            3: '3',
                            2: '2',
                            1: '1'
                        }
                        season = series_dict.get(season_number)

                        if int(match_data['gameData']['match_detail']['series']['id']) == int(season):

                            teams = match_data['gameData']['teams']['team']
                            for team in teams:
                                if team['id'] == team_id:
                                    squad = team['squad']
                                    for player in squad:
                                        player_id = player['id']
                                        player_name = player['name']

                                        if player_id not in [p['Player ID'] for p in roster]:
                                            player_details = {
                                                'Player ID': player_id,
                                                'Name': player_name,
                                                'Jersey Number': player.get('jersey', None),
                                                'Captain': player.get('captain', False),
                                                'Played': player.get('played', False),
                                                'Starter': player.get('starter', False),
                                                'Top Raider': player.get('top_raider', False),
                                                'Top Defender': player.get('top_defender', False)
                                            }

                                            roster.append(player_details)
                        else:
                            print("season not found.")

        roster_df = pd.DataFrame(roster)
        return roster_df

    def get_available_seasons(self) -> List[str]:
        """
        Get a list of available seasons in the dataset.

        Returns:
            List[str]: A list of season names.
        """
        return [folder for folder in os.listdir(self.base_path)
                if os.path.isdir(os.path.join(self.base_path, folder))]

    def get_matches_for_season(self, season: str) -> List[str]:
        """
        Get a list of match IDs for a specific season.

        Args:
            season (str): The season name.

        Returns:
            List[str]: A list of match IDs.
        """
        season_path = os.path.join(self.base_path, season)
        return [file.split('_ID_')[1].split('.')[0] for file in os.listdir(season_path) if file.endswith('.json')]

    # def get_match_data(self, season: str, match_id: str) -> Tuple[DataFrame, DataFrame, DataFrame, DataFrame, DataFrame, DataFrame]:
    #     """
    #     Get the full data for a specific match.
    #
    #     Args:
    #         season (str): The season name.
    #         match_id (str): The match ID.
    #
    #     Returns:
    #         Tuple[DataFrame, DataFrame, DataFrame, DataFrame, DataFrame, DataFrame]:
    #         A tuple containing match detail, teams, events, zones, team1, and team2 DataFrames.
    #     """
    #     season_path = os.path.join(self.base_path, season)
    #     file_name = next((f for f in os.listdir(season_path) if f.endswith(f'_ID_{match_id}.json')), None)
    #
    #     if not file_name:
    #         raise FileNotFoundError(f"No match file found for season {season} and match ID {match_id}")
    #
    #     file_path = os.path.join(season_path, file_name)
    #
    #     try:
    #         with open(file_path, 'r') as file:
    #             temp = json.load(file)
    #             match_detail = temp.get("match_detail", {})
    #             match_detail = match_detail.get("match_detail",{})
    #         flattened_match_detail = self._flatten_match_detail(match_detail)
    #         match_detail_df = pd.DataFrame([flattened_match_detail])
    #
    #         events_df = pd.DataFrame(temp.get("events", {}).get("event", []))
    #         zones_df = pd.DataFrame(temp.get("zones", {}).get("zone", []))
    #
    #         teams_data = temp.get("teams", {}).get("team", [])
    #         if len(teams_data) != 2:
    #             raise ValueError("Expected data for exactly two teams")
    #
    #         team1_df, team2_df = self._process_team_data(teams_data)
    #         teams_df = pd.DataFrame(teams_data)
    #
    #         return match_detail_df, teams_df, events_df, zones_df, team1_df, team2_df
    #
    #     except Exception as e:
    #         print(f"Error loading data from {file_path}: {str(e)}")
    #         return None, None, None, None, None, None
    def get_match_data(self, season: str, match_id: str) -> Tuple[
        DataFrame, DataFrame, DataFrame, DataFrame, DataFrame, DataFrame]:
        """
        Get the full data for a specific match.

        Args:
            season (str): The season name.
            match_id (str): The match ID.

        Returns:
            Tuple[DataFrame, DataFrame, DataFrame, DataFrame, DataFrame, DataFrame]:
            A tuple containing match detail, teams, events, zones, team1, and team2 DataFrames.
        """
        season_path = os.path.join(self.base_path, season)
        file_name = next((f for f in os.listdir(season_path) if f.endswith(f'_ID_{match_id}.json')), None)

        if not file_name:
            raise FileNotFoundError(f"No match file found for season {season} and match ID {match_id}")

        file_path = os.path.join(season_path, file_name)

        try:
            with open(file_path, 'r') as file:
                temp = json.load(file)

            # Check if the data is nested under 'gameData'
            if 'gameData' in temp:
                temp = temp['gameData']

            match_detail = temp.get("match_detail", {})
            flattened_match_detail = self._flatten_match_detail(match_detail)
            match_detail_df = pd.DataFrame([flattened_match_detail])

            events_df = pd.DataFrame(temp.get("events", {}).get("event", []))
            zones_df = pd.DataFrame(temp.get("zones", {}).get("zone", []))

            teams_data = temp.get("teams", {}).get("team", [])
            if len(teams_data) != 2:
                raise ValueError("Expected data for exactly two teams")

            team1_df, team2_df = self._process_team_data(teams_data)
            teams_df = pd.DataFrame(teams_data)

            return match_detail_df, teams_df, events_df, zones_df, team1_df, team2_df

        except Exception as e:
            print(f"Error loading data from {file_path}: {str(e)}")
            return None, None, None, None, None, None
    def get_match_events(self, season: str, match_id: str) -> DataFrame:
        """
        Get all events for a specific match.

        Args:
            season (str): The season name.
            match_id (str): The match ID.

        Returns:
            DataFrame: A DataFrame containing all events in the match.
        """
        _, _, events_df, _, _, _ = self.get_match_data(season, match_id)
        return events_df

    def _flatten_match_detail(self, match_detail: Dict[str, Any]) -> Dict[str, Any]:
        """
        Flatten nested dictionaries in match detail.

        Args:
            match_detail (Dict[str, Any]): The match detail dictionary.

        Returns:
            Dict[str, Any]: Flattened match detail dictionary.
        """
        flattened = {}
        for key, value in match_detail.items():
            if isinstance(value, dict):
                for subkey, subvalue in value.items():
                    flattened[f"{key}_{subkey}"] = subvalue
            elif isinstance(value, list):
                if key == "player_of_the_match":
                    flattened[f"{key}_id"] = value[0].get("id") if value else None
                    flattened[f"{key}_value"] = value[0].get("value") if value else None
                else:
                    flattened[key] = json.dumps(value)
            else:
                flattened[key] = value
        return flattened

    def _process_team_data(self, teams_data: List[Dict[str, Any]]) -> Tuple[DataFrame, DataFrame]:
        """
        Process team data into DataFrames.

        Args:
            teams_data (List[Dict[str, Any]]): List containing data for both teams.

        Returns:
            Tuple[DataFrame, DataFrame]: DataFrames for team1 and team2.
        """

        def process_squad(squad_data):
            processed_squad = []
            for player in squad_data:
                player_dict = {
                    'id': player['id'],
                    'name': player['name'],
                    'jersey': player.get('jersey', ''),
                    'played': player.get('played', False),
                    'captain': player.get('captain', False),
                    'on_court': player.get('on_court', False),
                    'starter': player.get('starter', False),
                    'red_card': player.get('red_card', False),
                    'yellow_card': player.get('yellow_card', False),
                    'green_card': player.get('green_card', False),
                    'red_card_count': player.get('red_card_count', 0),
                    'yellow_card_count': player.get('yellow_card_count', 0),
                    'green_card_count': player.get('green_card_count', 0),
                    'top_raider': player.get('top_raider', False),
                    'top_defender': player.get('top_defender', False),
                    'total_points': player.get('points', {}).get('total', 0),
                    'raid_points': player.get('points', {}).get('raid_points', {}).get('total', 0),
                    'tackle_points': player.get('points', {}).get('tackle_points', {}).get('total', 0),
                    'raids_total': player.get('raids', {}).get('total', 0),
                    'raids_successful': player.get('raids', {}).get('successful', 0),
                    'tackles_total': player.get('tackles', {}).get('total', 0),
                    'tackles_successful': player.get('tackles', {}).get('successful', 0),
                }
                for zone_type in ['strong_zones', 'weak_zones']:
                    for zone in player.get(zone_type, {}).get(zone_type.rstrip('s'), []):
                        player_dict[f"{zone_type}_zone_{zone['zone_id']}"] = zone['points']
                processed_squad.append(player_dict)
            return processed_squad

        team1_df = pd.DataFrame(process_squad(teams_data[0].get('squad', [])))
        team2_df = pd.DataFrame(process_squad(teams_data[1].get('squad', [])))

        for team_df, team_data in zip([team1_df, team2_df], teams_data):
            team_df['team_id'] = team_data['id']
            team_df['team_name'] = team_data['name']
            team_df['team_score'] = team_data['score']
            team_df['team_short_name'] = team_data['short_name']

        return team1_df, team2_df

# # Usage example

    # x = api.build_team_roster('4',9)
    # print(x)
    # x = api.get_team_matches('5','4')
    # print(x)

    # x = api.get_season_matches('10')
    # print(x)

    # x = api.get_team_info(4)
    # print(x)

    # player_stats, rvd,defend_df= api.get_player_info(322,9)
    # print(player_stats)
    # print(rvd)
    # print(defend_df)


    # x = api.get_pkl_standings(season=7, qualified=False)
    # print(x)
    # # print("-"*100)
    # # print(y)
    ###########################################
    # print("-" * 100, "test--", "-" * 100)
    #
    # x, y = api.get_pkl_standings(season=1, qualified=True)
    # print(x)
    # print("-" * 100)
    # print(y)
    #
    # print("-" * 100)

    # print(x.to_latex())

    # print(m)

    # df = api.match_data(season=4)
    # print(df)

    # for a season -> display all the match ids for that season along with some basic info about the match (matches overview)
    # for a given match-id -> get play-by-play data

    # for a given team in a given season return aggregated stats for that team
    # for a given team in a season, return the defender skills and raider skills for that team

    # return all matches played by that team

    # get team roster with player-ids
    # for a given player-id return aggregated stats about that player


# Example Usage of our API


## Instantiating our class

In [2]:
api = KabaddiDataAPI()


We begin our analysis one of two ways.

One either by getting the standings for a particular season, and the other by getting all the matches for a particular season.

In [19]:
x = api.get_pkl_standings(season=7, qualified=False)
x


,Group,Season,Team_Id,Team_Name,League_position,Matches_played,Wins,Lost,Tied,Draws,No Result,League_points,Score_diff,Qualified
0,Main,7,2,Dabang Delhi K.C.,1,22,15,4,3,0,0,85,66,True
1,Main,7,4,Bengal Warriorz,2,22,14,5,3,0,0,83,71,True
2,Main,7,30,UP Yoddhas,3,22,13,7,2,0,0,74,9,True
3,Main,7,5,U Mumba,4,22,12,8,2,0,0,72,47,True
4,Main,7,28,Haryana Steelers,5,22,13,8,1,0,0,71,15,True
5,Main,7,1,Bengaluru Bulls,6,22,11,10,1,0,0,64,16,True
6,Main,7,3,Jaipur Pink Panthers,7,22,9,11,2,0,0,58,-13,False
7,Main,7,6,Patna Pirates,8,22,8,13,1,0,0,51,29,False
8,Main,7,31,Gujarat Giants,9,22,7,13,2,0,0,51,18,False
9,Main,7,7,Puneri Paltan,10,22,7,12,3,0,0,48,-72,False


## Matches for a season

In [27]:
x = api.get_season_matches('10')
x


,Match Name,Match ID,Tour Name,Venue,Match_Outcome,Date,Result,Winning Margin,Gujarat Giants Score,Telugu Titans Score,U Mumba Score,U.P. Yoddhas Score,Tamil Thalaivas Score,Dabang Delhi K.C. Score,Bengaluru Bulls Score,Puneri Paltan Score,Jaipur Pink Panthers Score,Bengal Warriors Score,Patna Pirates Score,Haryana Steelers Score
0,Match 1,3029,"Pro Kabaddi League Season 10, 2023","EKA Arena by TransStadia, Ahmedabad",Gujarat Giants won by 6 Pts,2023-12-02T20:00+05:30,Gujarat Giants won by 6 Pts,6,38,32,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Match 2,3030,"Pro Kabaddi League Season 10, 2023","EKA Arena by TransStadia, Ahmedabad",U Mumba won by 3 Pts,2023-12-02T21:00+05:30,U Mumba won by 3 Pts,3,NaN,NaN,34,31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Match 3,3031,"Pro Kabaddi League Season 10, 2023","EKA Arena by TransStadia, Ahmedabad",Tamil Thalaivas won by 11 Pts,2023-12-03T20:00+05:30,Tamil Thalaivas won by 11 Pts,11,NaN,NaN,NaN,NaN,42,31,NaN,NaN,NaN,NaN,NaN,NaN
3,Match 4,3032,"Pro Kabaddi League Season 10, 2023","EKA Arena by TransStadia, Ahmedabad",Gujarat Giants won by 3 Pts,2023-12-03T21:00+05:30,Gujarat Giants won by 3 Pts,3,34,NaN,NaN,NaN,NaN,NaN,31,NaN,NaN,NaN,NaN,NaN
4,Match 5,3033,"Pro Kabaddi League Season 10, 2023","EKA Arena by TransStadia, Ahmedabad",Puneri Paltan won by 4 Pts,2023-12-04T20:00+05:30,Puneri Paltan won by 4 Pts,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,37,33,NaN,NaN,NaN
5,Match 6,3034,"Pro Kabaddi League Season 10, 2023","EKA Arena by TransStadia, Ahmedabad",Bengal Warriors won by 2 Pts,2023-12-04T21:00+05:30,Bengal Warriors won by 2 Pts,2,NaN,NaN,NaN,NaN,NaN,NaN,30,NaN,NaN,32,NaN,NaN
6,Match 7,3035,"Pro Kabaddi League Season 10, 2023","EKA Arena by TransStadia, Ahmedabad",Gujarat Giants won by 2 Pts,2023-12-05T20:00+05:30,Gujarat Giants won by 2 Pts,2,39,NaN,37,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Match 8,3036,"Pro Kabaddi League Season 10, 2023","EKA Arena by TransStadia, Ahmedabad",Patna Pirates won by 22 Pts,2023-12-06T21:00+05:30,Patna Pirates won by 22 Pts,22,NaN,28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,50,NaN
8,Match 9,3037,"Pro Kabaddi League Season 10, 2023","EKA Arena by TransStadia, Ahmedabad",U.P. Yoddhas won by 30 Pts,2023-12-06T21:00+05:30,U.P. Yoddhas won by 30 Pts,30,NaN,NaN,NaN,57,NaN,NaN,NaN,NaN,NaN,NaN,NaN,27
9,Match 10,3038,"Pro Kabaddi League Season 10, 2023","EKA Arena by TransStadia, Ahmedabad",Match Tied,2023-12-07T20:00+05:30,Match Tied,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,28,28,NaN,NaN


In [28]:
x = api.get_team_matches('5','4')
x


,Match Name,Match ID,Tour Name,Venue,Match_Outcome,Date,Result,Winning Margin
0,Match 10,295,"Pro Kabaddi League Season 5, 2017",Gachibowli Indoor Stadium,Bengal Warriors beat Telugu Titans (30-24),8/2/2017T9:00:00TPM,Bengal Warriors beat Telugu Titans (30-24),6
1,Match 16,301,"Pro Kabaddi League Season 5, 2017",Mankapur Indoor Stadium,Bengal Warriors beat U.P. Yoddha (40-20),8/6/2017T8:00:00TPM,Bengal Warriors beat U.P. Yoddha (40-20),20
2,Match 20,305,"Pro Kabaddi League Season 5, 2017",Mankapur Indoor Stadium,Bengaluru Bulls beat Bengal Warriors (31-25),8/9/2017T8:00:00TPM,Bengaluru Bulls beat Bengal Warriors (31-25),6
3,Match 28,313,"Pro Kabaddi League Season 5, 2017",The Arena by TransStadia,Puneri Paltan beat Bengal Warriors (34-17),8/15/2017T8:00:00TPM,Puneri Paltan beat Bengal Warriors (34-17),17
4,Match 33,318,"Pro Kabaddi League Season 5, 2017",The Arena by TransStadia,Match Tied,8/17/2017T9:00:00TPM,Match Tied,0
5,Match 41,326,"Pro Kabaddi League Season 5, 2017",Babu Banarasi Das Indoor Stadium,Bengal Warriors beat U.P. Yoddha (32-31),8/22/2017T9:00:00TPM,Bengal Warriors beat U.P. Yoddha (32-31),1
6,Match 46,331,"Pro Kabaddi League Season 5, 2017","â€˜Dome, NSCI SVP Stadiumâ€™",Match Tied,8/25/2017T9:00:00TPM,Match Tied,0
7,Match 49,334,"Pro Kabaddi League Season 5, 2017","â€˜Dome, NSCI SVP Stadiumâ€™",Bengal Warriors beat Bengaluru Bulls (32-26),8/27/2017T8:00:00TPM,Bengal Warriors beat Bengaluru Bulls (32-26),6
8,Match 56,341,"Pro Kabaddi League Season 5, 2017",Netaji Indoor Stadium,Bengal Warriors beat Patna Pirates (41-38),9/1/2017T8:00:00TPM,Bengal Warriors beat Patna Pirates (41-38),3
9,Match 58,343,"Pro Kabaddi League Season 5, 2017",Netaji Indoor Stadium,Match Tied,9/2/2017T9:00:00TPM,Match Tied,0


In [29]:
x = api.build_team_roster('4',9)
x


,Player ID,Name,Jersey Number,Captain,Played,Starter,Top Raider,Top Defender
0,143,Maninder Singh,9,True,True,True,True,False
1,5114,Vaibhav Garje,15,False,True,True,False,False
2,4978,Balaji D,2,False,True,True,False,False
3,3151,Manoj Gowda,27,False,True,True,False,False
4,106,Shrikant Jadhav,22,False,True,True,False,False
5,3103,Shubham Shinde,69,False,True,True,False,True
6,161,Girish Maruti Ernak,8,False,True,True,False,False
7,146,Surender Nada,1,False,True,False,False,False
8,41,Deepak Hooda,5,False,False,False,False,False
9,4195,Sakthivel R,7,False,True,False,False,False


In [32]:
df_rank, df_value, df_per_match, standings_df, matches_df = api.get_team_info(4,9)
df_per_match


{'Group': 'Main', 'match_id': 2897, 'date': '2022-10-08', 'teama_id': 4, 'result': 'L', 'teama_short_name': 'KOL', 'teama_score': 33, 'teamb_id': 28, 'teamb_short_name': 'HAR', 'teamb_score': 41, 'match_result': 'Haryana Steelers Won by 8 Pts'}
{'Group': 'Main', 'match_id': 2899, 'date': '2022-10-09', 'teama_id': 8, 'result': 'W', 'teama_short_name': 'HYD', 'teama_score': 25, 'teamb_id': 4, 'teamb_short_name': 'KOL', 'teamb_score': 45, 'match_result': 'Bengal Warriorz Won by 20 Pts'}
{'Group': 'Main', 'match_id': 2905, 'date': '2022-10-12', 'teama_id': 1, 'result': 'W', 'teama_short_name': 'BLR', 'teama_score': 33, 'teamb_id': 4, 'teamb_short_name': 'KOL', 'teamb_score': 42, 'match_result': 'Bengal Warriorz Won by 9 Pts'}
{'Group': 'Main', 'match_id': 2912, 'date': '2022-10-15', 'teama_id': 4, 'result': 'W', 'teama_short_name': 'KOL', 'teama_score': 54, 'teamb_id': 6, 'teamb_short_name': 'PAT', 'teamb_score': 26, 'match_result': 'Bengal Warriorz Won by 28 Pts'}
{'Group': 'Main', 'match

,season,team_id,team_name,team-average-raid-points_per-match,team-avg-points-scored_per-match,team-all-outs-conceded_per-match,team-successful-tackle-percent_per-match,team-super-raid_per-match,team-raid_per-match,team-successful-raid-percent_per-match,team-dod-raid-points_per-match,team-super-tackles_per-match,team-total-touch-points-ext_per-match,team-total-bonus--points-ext_per-match,team-raid-points_per-match,team-successful-raids_per-match,team-total-points-conceded_per-match,team-tackle-points_per-match,team-total-points_per-match,team-successful-tackles-per-match_per-match,team-all-outs-inflicted_per-match,team-average-tackle-points_per-match,team-successful-tackles_per-match
83,9.0,4,Bengal Warriors,1.01,1.661,1.455,1.698,1.0,42.0,1.86,2.727,0.455,16.227,6.0,22.227,17.182,37.091,9.727,36.545,0.421,1.364,0.442,9.273


In [37]:
#api = KabaddiAPI("./MatchData_pbp")
match_detail_df, teams_df, events_df, zones_df, team1_df, team2_df = api.get_match_data('Season_PKL_Season_4_2016', '194')


In [38]:
match_detail_df


,match_id,match_number,clock_minutes,clock_seconds,date,start_time,gmtoffset,result_outcome,result_value,result_winning_method,result_winning_team,result_winning_team_id,player_of_the_match_id,player_of_the_match_value,series_id,series_name,series_short_name,series_parent_series_id,series_parent_series_name,status_id,status,toss_winner,toss_selection,venue_id,venue_name
0,194,Match 13,39,56,7/2/2016,20:00,+05:30,,Dabang Delhi K.C. beat Bengaluru Bulls (32-24),,Dabang Delhi K.C.,2,165,Sachin Shingade,4,"Pro Kabaddi League Season 4, 2016",league,1,Pro Kabaddi League,5,Full Time,1,Choice of court :Right,3,"Sawai Mansingh Stadium, Jaipur"


In [39]:
events_df


,event_no,event,event_id,event_text,raider_id,raiding_team_id,defender_id,defending_team_id,raid_points,defending_points,clock,score,player_id,substituted_by,team_id,reason,reason_id
0,1,Empty Raid,3,Meraj Sheykh Empty Raid,251.0,2.0,NaN,1.0,0.0,0.0,00:01,"[0, 0]",NaN,NaN,NaN,NaN,NaN
1,2,Empty Raid,3,Rohit Kumar Empty Raid,326.0,1.0,NaN,2.0,0.0,0.0,00:26,"[0, 0]",NaN,NaN,NaN,NaN,NaN
2,3,Unsuccessful Raid,2,Kashiling Adake Unsuccessful Raid,58.0,2.0,146.0,1.0,0.0,1.0,00:41,"[0, 1]",NaN,NaN,NaN,NaN,NaN
3,4,Successful Raid,1,Vinod Kumar raids successfully,387.0,1.0,NaN,2.0,1.0,0.0,01:06,"[0, 2]",NaN,NaN,NaN,NaN,NaN
4,5,Empty Raid,3,Selvamani K Empty Raid,264.0,2.0,NaN,1.0,0.0,0.0,01:37,"[0, 2]",NaN,NaN,NaN,NaN,NaN
5,6,Empty Raid,3,Rohit Kumar Empty Raid,326.0,1.0,NaN,2.0,0.0,0.0,02:05,"[0, 2]",NaN,NaN,NaN,NaN,NaN
6,7,Empty Raid,3,Meraj Sheykh Empty Raid,251.0,2.0,NaN,1.0,0.0,0.0,02:34,"[0, 2]",NaN,NaN,NaN,NaN,NaN
7,8,Empty Raid,3,Yogesh Hooda Empty Raid,245.0,1.0,NaN,2.0,0.0,0.0,03:02,"[0, 2]",NaN,NaN,NaN,NaN,NaN
8,9,Successful Raid,1,Selvamani K raids successfully,264.0,2.0,NaN,1.0,1.0,0.0,03:25,"[1, 2]",NaN,NaN,NaN,NaN,NaN
9,10,Unsuccessful Raid,2,Rohit Kumar Unsuccessful Raid,326.0,1.0,360.0,2.0,0.0,1.0,03:52,"[2, 2]",NaN,NaN,NaN,NaN,NaN


In [40]:
team1_df


,id,name,jersey,played,captain,on_court,starter,red_card,yellow_card,green_card,red_card_count,yellow_card_count,green_card_count,top_raider,top_defender,total_points,raid_points,tackle_points,raids_total,raids_successful,tackles_total,tackles_successful,strong_zones_zone_1,strong_zones_zone_2,strong_zones_zone_3,strong_zones_zone_4,strong_zones_zone_5,strong_zones_zone_6,strong_zones_zone_7,strong_zones_zone_8,strong_zones_zone_9,strong_zones_zone_10,strong_zones_zone_11,weak_zones_zone_1,weak_zones_zone_2,weak_zones_zone_3,weak_zones_zone_4,weak_zones_zone_5,weak_zones_zone_6,weak_zones_zone_7,weak_zones_zone_8,weak_zones_zone_9,weak_zones_zone_10,weak_zones_zone_11,team_id,team_name,team_score,team_short_name
0,58,Kashiling Adake,11,True,True,False,True,False,False,False,0,0,0,False,False,2,2,0,14,1,1,0,0,0,0,0,2,0,0,0,0,0,0,0,1,1,0,1,1,0,0,0,0,0,2,Dabang Delhi K.C.,32,DEL
1,162,D. Suresh Kumar,1,True,False,True,True,False,False,False,0,0,0,False,False,1,0,1,0,0,4,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,Dabang Delhi K.C.,32,DEL
2,165,Sachin Shingade,17,True,False,False,True,False,False,False,0,0,0,False,True,7,0,7,0,0,8,6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,Dabang Delhi K.C.,32,DEL
3,264,Selvamani K,10,True,False,True,True,False,False,False,0,0,0,False,False,2,2,0,8,2,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,1,0,0,0,0,0,0,0,2,Dabang Delhi K.C.,32,DEL
4,251,Meraj Sheykh,2,True,False,True,True,False,False,False,0,0,0,True,False,8,7,1,19,7,2,1,0,0,1,1,0,3,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,2,Dabang Delhi K.C.,32,DEL
5,152,D. Gopu,5,True,False,True,True,False,False,False,0,0,0,False,False,5,0,5,0,0,5,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,Dabang Delhi K.C.,32,DEL
6,360,Sanket Chavan,63,True,False,True,True,False,False,False,0,0,0,False,False,2,0,2,0,0,3,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,Dabang Delhi K.C.,32,DEL
7,301,Hadi Tajik,44,True,False,False,False,False,False,False,0,0,0,False,False,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,Dabang Delhi K.C.,32,DEL
8,34,Bhupender Singh,6,True,False,False,False,False,False,False,0,0,0,False,False,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,Dabang Delhi K.C.,32,DEL
9,147,Prashant Chavan,18,True,False,False,False,False,False,False,0,0,0,False,False,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,Dabang Delhi K.C.,32,DEL


In [23]:
player_stats, rvd,df1= api.get_player_info(322,9)
player_stats


,Unique_ID,season,player_id,player-total-points_player_name,player-total-points_match_played,player-total-points_position_id,player-total-points_position_name,player-total-points_team_name,team_id,player-total-points_team_full_name,player-super-tackles_rank,player-raid-points_rank,player-super-raids_rank,high-5s_rank,player-tackle-points_rank,player-avg-tackle-points_rank,player-dod-raid-points_rank,player-total-points_rank,player-successful-tackles_rank,player-successful-raids_rank,super-10s_rank,player-super-tackles_value,player-raid-points_value,player-super-raids_value,high-5s_value,player-tackle-points_value,player-avg-tackle-points_value,player-dod-raid-points_value,player-total-points_value,player-successful-tackles_value,player-successful-raids_value,super-10s_value,player-super-tackles_points_per_match,player-raid-points_points_per_match,player-super-raids_points_per_match,high-5s_points_per_match,player-tackle-points_points_per_match,player-avg-tackle-points_points_per_match,player-dod-raid-points_points_per_match,player-total-points_points_per_match,player-successful-tackles_points_per_match,player-successful-raids_points_per_match,super-10s_points_per_match
749,team_id_8|Season9|Surjeet Singh,9,322,Surjeet Singh,13,0,Defender - Right Cover,HYD,8,Telugu Titans,43.0,105.0,NaN,NaN,59.0,62.0,NaN,108,58.0,103.0,NaN,2.0,2.0,NaN,NaN,17.0,1.31,NaN,19,15.0,1.0,NaN,0.15,0.15,NaN,NaN,1.31,0.1,NaN,1.46,1.15,0.08,NaN


In [25]:
rvd


,season,team_name,player-id-pkdc-sanitised,Raider Name,Number of Defenders,Total Raids,% of Raids,Empty,Successful,player-id
2457,PKL-09,Telugu Titans,322,Surjeet Singh,4,200%,18.00%,100.00%,NaN,322
2458,PKL-09,Telugu Titans,322,Surjeet Singh,5,100%,9.00%,100.00%,NaN,322
2459,PKL-09,Telugu Titans,322,Surjeet Singh,6,100%,9.00%,100.00%,NaN,322
2460,PKL-09,Telugu Titans,322,Surjeet Singh,7,700%,64.00%,71.40%,14.30%,322


In [24]:
df1


,Unique_ID,season,team_id,Team,player_id_copy_backup,Defender Name,Total Tackles,Successful Tackles,Defender Success rate,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,player_id
931,team_id_8|9|Surjeet Singh,9,8,Telugu Titans,322,Surjeet Singh,48,15,31%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,322


,Match Name,Match ID,Tour Name,Venue,Match_Outcome,Date,Result,Winning Margin,Gujarat Giants Score,Telugu Titans Score,U Mumba Score,U.P. Yoddhas Score,Tamil Thalaivas Score,Dabang Delhi K.C. Score,Bengaluru Bulls Score,Puneri Paltan Score,Jaipur Pink Panthers Score,Bengal Warriors Score,Patna Pirates Score,Haryana Steelers Score
0,Match 1,3029,"Pro Kabaddi League Season 10, 2023","EKA Arena by TransStadia, Ahmedabad",Gujarat Giants won by 6 Pts,2023-12-02T20:00+05:30,Gujarat Giants won by 6 Pts,6,38,32,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Match 2,3030,"Pro Kabaddi League Season 10, 2023","EKA Arena by TransStadia, Ahmedabad",U Mumba won by 3 Pts,2023-12-02T21:00+05:30,U Mumba won by 3 Pts,3,NaN,NaN,34,31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Match 3,3031,"Pro Kabaddi League Season 10, 2023","EKA Arena by TransStadia, Ahmedabad",Tamil Thalaivas won by 11 Pts,2023-12-03T20:00+05:30,Tamil Thalaivas won by 11 Pts,11,NaN,NaN,NaN,NaN,42,31,NaN,NaN,NaN,NaN,NaN,NaN
3,Match 4,3032,"Pro Kabaddi League Season 10, 2023","EKA Arena by TransStadia, Ahmedabad",Gujarat Giants won by 3 Pts,2023-12-03T21:00+05:30,Gujarat Giants won by 3 Pts,3,34,NaN,NaN,NaN,NaN,NaN,31,NaN,NaN,NaN,NaN,NaN
4,Match 5,3033,"Pro Kabaddi League Season 10, 2023","EKA Arena by TransStadia, Ahmedabad",Puneri Paltan won by 4 Pts,2023-12-04T20:00+05:30,Puneri Paltan won by 4 Pts,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,37,33,NaN,NaN,NaN
5,Match 6,3034,"Pro Kabaddi League Season 10, 2023","EKA Arena by TransStadia, Ahmedabad",Bengal Warriors won by 2 Pts,2023-12-04T21:00+05:30,Bengal Warriors won by 2 Pts,2,NaN,NaN,NaN,NaN,NaN,NaN,30,NaN,NaN,32,NaN,NaN
6,Match 7,3035,"Pro Kabaddi League Season 10, 2023","EKA Arena by TransStadia, Ahmedabad",Gujarat Giants won by 2 Pts,2023-12-05T20:00+05:30,Gujarat Giants won by 2 Pts,2,39,NaN,37,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Match 8,3036,"Pro Kabaddi League Season 10, 2023","EKA Arena by TransStadia, Ahmedabad",Patna Pirates won by 22 Pts,2023-12-06T21:00+05:30,Patna Pirates won by 22 Pts,22,NaN,28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,50,NaN
8,Match 9,3037,"Pro Kabaddi League Season 10, 2023","EKA Arena by TransStadia, Ahmedabad",U.P. Yoddhas won by 30 Pts,2023-12-06T21:00+05:30,U.P. Yoddhas won by 30 Pts,30,NaN,NaN,NaN,57,NaN,NaN,NaN,NaN,NaN,NaN,NaN,27
9,Match 10,3038,"Pro Kabaddi League Season 10, 2023","EKA Arena by TransStadia, Ahmedabad",Match Tied,2023-12-07T20:00+05:30,Match Tied,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,28,28,NaN,NaN
